In [1]:
import pandas as pd
import numpy as np
import requests
import gmaps
import matplotlib.pyplot as plt
from census import Census
from us import states
from api_keys import census_key, gov_key, gkey

In [2]:
gmaps.configure(api_key=gkey)

In [3]:
c = Census(census_key, year=2016)

In [4]:
census_data = c.acs5.get(("NAME", "B19013_001E", "B01003_001E", "B01002_001E",
                          "B19301_001E",
                          "B17001_002E","B23025_005E"), {'for': 'county:*'})

In [5]:
census_data

[{'NAME': 'Autauga County, Alabama',
  'B19013_001E': 53099.0,
  'B01003_001E': 55049.0,
  'B01002_001E': 37.8,
  'B19301_001E': 26168.0,
  'B17001_002E': 6697.0,
  'B23025_005E': 1437.0,
  'state': '01',
  'county': '001'},
 {'NAME': 'Baldwin County, Alabama',
  'B19013_001E': 51365.0,
  'B01003_001E': 199510.0,
  'B01002_001E': 42.3,
  'B19301_001E': 28069.0,
  'B17001_002E': 25551.0,
  'B23025_005E': 5887.0,
  'state': '01',
  'county': '003'},
 {'NAME': 'Barbour County, Alabama',
  'B19013_001E': 33956.0,
  'B01003_001E': 26614.0,
  'B01002_001E': 38.7,
  'B19301_001E': 17249.0,
  'B17001_002E': 6235.0,
  'B23025_005E': 1323.0,
  'state': '01',
  'county': '005'},
 {'NAME': 'Bibb County, Alabama',
  'B19013_001E': 39776.0,
  'B01003_001E': 22572.0,
  'B01002_001E': 40.2,
  'B19301_001E': 18988.0,
  'B17001_002E': 3390.0,
  'B23025_005E': 643.0,
  'state': '01',
  'county': '007'},
 {'NAME': 'Blount County, Alabama',
  'B19013_001E': 46212.0,
  'B01003_001E': 57704.0,
  'B01002_001E

In [6]:
# Convert to DataFrame
census_pd = pd.DataFrame(census_data)

# Column Reordering
census_pd = census_pd.rename(columns={"B01003_001E": "Population",
                                      "B01002_001E": "Median Age",
                                      "B19013_001E": "Household Income",
                                      "B19301_001E": "Per Capita Income",
                                      "B17001_002E": "Poverty Count",
                                      "B23025_005E": "Unemployment",
                                      "NAME": "Name", "state": "State", "county":"County"})

# Add in Poverty Rate (Poverty Count / Population)
census_pd["Poverty Rate"] = census_pd["Poverty Count"].astype(
        int) / census_pd["Population"].astype(int)

census_pd['Unemployment Rate'] = census_pd['Unemployment'].astype(int) / census_pd['Population'].astype(int)
# Final DataFrame
census_pd = census_pd[["State", "County", "Population", "Median Age", "Household Income",
                       "Per Capita Income", "Poverty Count", "Poverty Rate",'Unemployment Rate']]

# Visualize
print(len(census_pd))
census_pd.head()

3220


,State,County,Population,Median Age,Household Income,Per Capita Income,Poverty Count,Poverty Rate,Unemployment Rate
0,01,001,55049.0,37.8,53099.0,26168.0,6697.0,0.121655,0.026104
1,01,003,199510.0,42.3,51365.0,28069.0,25551.0,0.128069,0.029507
2,01,005,26614.0,38.7,33956.0,17249.0,6235.0,0.234275,0.049711
3,01,007,22572.0,40.2,39776.0,18988.0,3390.0,0.150186,0.028487
4,01,009,57704.0,40.8,46212.0,21033.0,9441.0,0.163611,0.023690


In [ ]:
# census_pd[census_pd['Household Income']<0].shape

# zip_code_locs = pd.read_csv('~/Downloads/zip_code_locs.csv', dtype = {'ZIP':str, 'LAT':float, 'LNG':float})\
# .rename(columns = {'ZIP':'Zip Code'})

# zip_code_locs.head()

In [ ]:
# census_w_loc = pd.merge(census_pd, zip_code_locs, how = 'left', on = 'Zip Code')

In [ ]:
# census_w_loc.head()

In [3]:
zip_code_info = pd.read_csv('/Users/seantibbitts/Downloads/US/US.txt', sep = '\t', header=None, dtype={1:str},
                            names=['Country','zipcode','City','State','State Code','Unk1','Unk2','Unk3','Unk4','Lat','Lng','Unk5'])

In [4]:
zip_code_info.head()

,Country,zipcode,City,State,State Code,Unk1,Unk2,Unk3,Unk4,Lat,Lng,Unk5
0,US,99553,Akutan,Alaska,AK,Aleutians East,13.0,NaN,NaN,54.1430,-165.7854,1.0
1,US,99571,Cold Bay,Alaska,AK,Aleutians East,13.0,NaN,NaN,55.1858,-162.7211,1.0
2,US,99583,False Pass,Alaska,AK,Aleutians East,13.0,NaN,NaN,54.8542,-163.4113,1.0
3,US,99612,King Cove,Alaska,AK,Aleutians East,13.0,NaN,NaN,55.0628,-162.3056,1.0
4,US,99661,Sand Point,Alaska,AK,Aleutians East,13.0,NaN,NaN,55.3192,-160.4914,1.0


In [5]:
zip_code_info.dtypes

Country        object
zipcode        object
City           object
State          object
State Code     object
Unk1           object
Unk2          float64
Unk3           object
Unk4          float64
Lat           float64
Lng           float64
Unk5          float64
dtype: object

In [ ]:
# locations = census_w_loc[['LAT','LNG']]

# fig = gmaps.figure()
# fig.add_layer(gmaps.heatmap_layer(locations, max_intensity = 15, weights = census_w_loc['Unemployment Rate'],
#                                   dissipating = False, point_radius = 1))
# fig

In [23]:
import gmaps.geojson_geometries

In [24]:
counties_geojson = gmaps.geojson_geometries.load_geometry('us-counties')

In [12]:
fig = gmaps.figure()
counties_layer = gmaps.geojson_layer(counties_geojson)
fig.add_layer(counties_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [13]:
counties_geojson['features'][:10]

[{'type': 'Feature',
  'properties': {'GEO_ID': '0500000US01001',
   'STATE': '01',
   'COUNTY': '001',
   'NAME': 'Autauga',
   'LSAD': 'County',
   'CENSUSAREA': 594.436},
  'geometry': {'type': 'Polygon',
   'coordinates': [[[-86.496774, 32.344437],
     [-86.717897, 32.402814],
     [-86.814912, 32.340803],
     [-86.890581, 32.502974],
     [-86.917595, 32.664169],
     [-86.71339, 32.661732],
     [-86.714219, 32.705694],
     [-86.413116, 32.707386],
     [-86.411172, 32.409937],
     [-86.496774, 32.344437]]]}},
 {'type': 'Feature',
  'properties': {'GEO_ID': '0500000US01009',
   'STATE': '01',
   'COUNTY': '009',
   'NAME': 'Blount',
   'LSAD': 'County',
   'CENSUSAREA': 644.776},
  'geometry': {'type': 'Polygon',
   'coordinates': [[[-86.577799, 33.765316],
     [-86.759144, 33.840617],
     [-86.953664, 33.815297],
     [-86.954305, 33.844862],
     [-86.96296, 33.844865],
     [-86.963358, 33.858221],
     [-86.924387, 33.909222],
     [-86.793914, 33.952059],
     [-86.685

In [14]:
county2unemploy = {(row['State'],row['County']):row['Unemployment Rate'] for i, row in census_pd.iterrows()}

In [15]:
# We will need to scale the UR values to lie between 0 and 1
min_ur = min(county2unemploy.values())
max_ur = max(county2unemploy.values())
ur_range = max_ur - min_ur

In [26]:
from matplotlib.cm import viridis
from matplotlib.colors import to_hex

def calculate_color(ur):
    # make ur a number between 0 and 1
    normalized_ur = (ur - min_ur) / ur_range
    
    # invert unemployment rate so high unemployment gets a dark color
    inverse_ur = 1.0 - normalized_ur
    
    # transform the unemployment rate to a matplotlib color
    mpl_color = viridis(inverse_ur)

    # transform from a matplotlib color to a valid CSS color
    gmaps_color = to_hex(mpl_color, keep_alpha=False)

    return gmaps_color

In [17]:
colors = []

for feature in counties_geojson['features']:
    state = feature['properties']['STATE']
    county = feature['properties']['COUNTY']
    try:
        ur = county2unemploy[(state,county)]
        color = calculate_color(ur)
    except KeyError:
        # no UR for that county: return default color
        color = (0, 0, 0, 0.3)
    colors.append(color)

In [18]:
fig = gmaps.figure()
counties_layer = gmaps.geojson_layer(counties_geojson, fill_color=colors, stroke_color=colors, fill_opacity=0.8)
fig.add_layer(counties_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Solar Installs

In [ ]:
# url = f'https://developer.nrel.gov/api/solar/open_pv/installs/index?api_key={gov_key}'

In [ ]:
# requests.get(url)

In [6]:
open_pv = pd.read_csv('/Users/seantibbitts/Downloads/openpv_all.csv', dtype = {'zipcode':str})

/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (2,5,8,12,13,14,15,16,20,23,25,26,27,28,32,50) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
open_pv.head()

,state,date_installed,incentive_prog_names,type,size_kw,appraised,zipcode,install_type,installer,cost_per_watt,...,new_constr,effic_1,cust_county,tracking,inv_model_clean,mod_cost_real,inv_cost_real,bos_powerclerk_real,permitting_real,3rdparty
0,TX,12/12/2013,Austin Energy,uir,10.560,NaN,78702,residential,NaN,2.897777,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,TX,07/11/2014,Austin Energy,uir,12.720,NaN,78702,residential,NaN,3.144654,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,TX,01/17/2014,Austin Energy,uir,5.400,NaN,78702,residential,NaN,4.197330,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,IN,10/06/2017,NaN,uir,6.785,NaN,47403,Residential,Whole Sun Designs,2.590000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,IN,10/06/2017,NaN,uir,4.720,NaN,47408,Residential,Whole Sun Designs,2.570000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
open_pv.shape

(1020945, 81)

In [9]:
open_pv.columns

Index(['state', 'date_installed', 'incentive_prog_names', 'type', 'size_kw',
       'appraised', 'zipcode', 'install_type', 'installer', 'cost_per_watt',
       'cost', 'lbnl_tts_version_year', 'lbnl_tts', 'city', 'utility_clean',
       'tech_1', 'model1_clean', 'county', 'annual_PV_prod',
       'annual_insolation', 'rebate', 'sales_tax_cost', 'tilt1',
       'tracking_type', 'azimuth1', 'manuf2_clean', 'manuf3_clean',
       'manuf1_clean', 'inv_man_clean', 'reported_annual_energy_prod',
       'incentivetype', 'year_app_implied', 'year', 'npv_fit_real',
       'application_implied', 'npv_pbi_real', 'other_incentive',
       'appraised_cluster', 'inflation', 'other_incentive_real',
       'zip_available', 'cust_city', 'pbi', 'pbi_real', 'pbi_length',
       'application', 'fit_length', 'fit_rate', 'fit_payment',
       '_3rdparty_implied', 'utility', 'install_price_real_w', 'install_price',
       'installer_clean', 'manuf1_', 'inverter_reported', 'rebate_real',
       'model1', '_3

In [10]:
open_pv['date_installed'].value_counts(dropna = False).sort_index()

01/01/1998       2
01/01/1999       6
01/01/2000      22
01/01/2001      10
01/01/2002      10
01/01/2003       5
01/01/2004      24
01/01/2005      39
01/01/2006      35
01/01/2007      65
01/01/2008     150
01/01/2009     290
01/01/2010      25
01/01/2011      35
01/01/2012      69
01/01/2013      15
01/01/2014      13
01/01/2015      91
01/01/2016       3
01/01/2017       1
01/02/1999       1
01/02/2002      18
01/02/2003       8
01/02/2004       1
01/02/2005       1
01/02/2006      10
01/02/2007      10
01/02/2008      50
01/02/2009     148
01/02/2010       4
              ... 
12/30/2005      59
12/30/2006      14
12/30/2007      16
12/30/2008     145
12/30/2009     373
12/30/2010     504
12/30/2011     492
12/30/2012      19
12/30/2013     654
12/30/2014    1127
12/30/2015    1065
12/30/2016       2
12/31/1969     119
12/31/1999       2
12/31/2001      35
12/31/2002      17
12/31/2003      15
12/31/2004      47
12/31/2005      12
12/31/2006      23
12/31/2007      88
12/31/2008  

In [11]:
open_pv['date_installed_dt'] = pd.to_datetime(open_pv['date_installed'])

In [12]:
open_pv['date_installed_dt'].value_counts(dropna = False).sort_index()

1909-07-07      4
1969-12-31    119
1983-06-01      1
1983-11-14      1
1984-12-01      1
1986-12-01      1
1988-08-23      1
1992-12-01      1
1994-01-25      1
1994-02-01      1
1994-03-08      2
1994-03-16      2
1994-03-31      1
1994-06-01      1
1994-10-07      1
1994-12-01      1
1994-12-30      1
1995-03-03      1
1995-06-01      1
1995-06-30      1
1995-07-05      2
1995-07-26      1
1995-07-27      1
1995-07-28      1
1995-08-01      1
1995-08-02      1
1995-08-10      1
1995-09-01      1
1995-10-27      1
1995-12-30      1
             ... 
2019-01-10      2
2019-01-11      1
2019-01-15      1
2019-01-16      1
2019-01-18      1
2019-01-21      1
2019-01-23      1
2019-01-25      1
2019-01-29      1
2019-02-01      1
2019-02-05      1
2019-02-06      2
2019-02-08      1
2019-02-11      3
2019-02-12      1
2019-02-13      1
2019-02-20      1
2019-02-22      1
2019-02-25      1
2019-02-26      1
2019-02-27      2
2019-03-01      1
2019-03-05      1
2019-03-13      1
2019-03-15

In [26]:
# Probably want to limit to more recent installs
open_pv1 = open_pv[open_pv['date_installed_dt'] > '2016'].copy()

In [27]:
open_pv1.shape

(975, 82)

In [28]:
# open_pv2 = open_pv[open_pv['zipcode'].notnull()].copy()
# Above not necessary after limiting to recent
open_pv2 = open_pv1.copy()

In [29]:
open_pv2.dtypes

state                                  object
date_installed                         object
incentive_prog_names                   object
type                                   object
size_kw                               float64
appraised                              object
zipcode                                object
install_type                           object
installer                              object
cost_per_watt                         float64
cost                                  float64
lbnl_tts_version_year                 float64
lbnl_tts                               object
city                                   object
utility_clean                          object
tech_1                                 object
model1_clean                           object
county                                 object
annual_PV_prod                        float64
annual_insolation                     float64
rebate                                 object
sales_tax_cost                    

In [30]:
open_pv_w_loc = pd.merge(open_pv2, zip_code_info, how = 'left', on='zipcode')

In [31]:
open_pv_w_loc.head()

,state,date_installed,incentive_prog_names,type,size_kw,appraised,zipcode,install_type,installer,cost_per_watt,...,City,State,State Code,Unk1,Unk2,Unk3,Unk4,Lat,Lng,Unk5
0,IN,10/06/2017,NaN,uir,6.785,NaN,47403,Residential,Whole Sun Designs,2.59,...,Bloomington,Indiana,IN,Monroe,105.0,NaN,NaN,39.1263,-86.5769,4.0
1,IN,10/06/2017,NaN,uir,4.720,NaN,47408,Residential,Whole Sun Designs,2.57,...,Bloomington,Indiana,IN,Monroe,105.0,NaN,NaN,39.2303,-86.4692,4.0
2,IN,10/13/2017,NaN,uir,12.390,NaN,47429,Residential,Whole Sun Designs,2.60,...,Ellettsville,Indiana,IN,Monroe,105.0,NaN,NaN,39.2545,-86.6196,4.0
3,IN,10/17/2017,NaN,uir,6.700,NaN,47401,Residential,Whole Sun Designs,2.28,...,Bloomington,Indiana,IN,Monroe,105.0,NaN,NaN,39.1401,-86.5083,4.0
4,IN,10/23/2017,NaN,uir,4.130,NaN,47404,Residential,Whole Sun Designs,2.62,...,Bloomington,Indiana,IN,Monroe,105.0,NaN,NaN,39.1950,-86.5757,4.0


In [32]:
# Some null lats and longs, but not many
# (None after limiting to 2016+)
open_pv_w_loc[open_pv_w_loc['Lat'].isnull()].shape

(0, 93)

In [33]:
# Drop out null locations
open_pv_w_loc2 = open_pv_w_loc[open_pv_w_loc['Lat'].notnull() & open_pv_w_loc['Lng'].notnull()].copy()

In [34]:
min_size = open_pv_w_loc2['size_kw'].min()
max_size = open_pv_w_loc2['size_kw'].max()
size_range = max_size - min_size
open_pv_w_loc2['size_normed'] = (10 * (open_pv_w_loc2['size_kw'] - min_size)/size_range + 1).astype(int)

In [35]:
fig = gmaps.figure()
install_locs = open_pv_w_loc2[['Lat','Lng']]
size = [int(x) for i, x in open_pv_w_loc2['size_normed'].iteritems()]

In [36]:
install_layer = gmaps.symbol_layer(install_locs, fill_opacity=.5, stroke_opacity = .5)

In [37]:
fig.add_layer(install_layer)

In [38]:
fig

Figure(layout=FigureLayout(height='420px'))

In [132]:
# installs = pd.DataFrame(results['result'])

# installs.shape

# installs.head()

# zip_code_locs.head()

# installs_w_loc = pd.merge(installs, zip_code_locs, how = 'left', left_on = 'zipcode', right_on = 'Zip Code')

# installs_w_loc.head()

# min_size = installs_w_loc['size_kw'].min()
# max_size = installs_w_loc['size_kw'].max()
# size_range = max_size - min_size
# installs_w_loc['size_normed'] = (10 * (installs_w_loc['size_kw'] - min_size)/size_range + 1).astype(int)

# installs_w_loc['size_normed'].value_counts(dropna = False)

# fig = gmaps.figure()
# install_locs = installs_w_loc[['LAT','LNG']]
# size = [int(x) for i, x in installs_w_loc['size_normed'].iteritems()]
# fig.add_layer(gmaps.symbol_layer(install_locs, scale = size, fill_opacity=.5, stroke_opacity = .5))
# fig

### Sunniness

In [13]:
sunny = pd.read_csv('/Users/seantibbitts/Downloads/North America Land Data Assimilation System (NLDAS) Daily Sunlight (KJm²) (1979-2011) (1).txt',
                    sep = '\t', dtype = {'County Code':str}, encoding = 'ISO-8859-1')

In [14]:
sunny

,Notes,County,County Code,Avg Daily Sunlight (KJ/m²)
0,NaN,"Autauga County, AL",01001,17103.72
1,NaN,"Baldwin County, AL",01003,17660.59
2,NaN,"Barbour County, AL",01005,17315.13
3,NaN,"Bibb County, AL",01007,16904.87
4,NaN,"Blount County, AL",01009,16393.63
5,NaN,"Bullock County, AL",01011,17296.02
6,NaN,"Butler County, AL",01013,17323.69
7,NaN,"Calhoun County, AL",01015,16447.09
8,NaN,"Chambers County, AL",01017,16997.01
9,NaN,"Cherokee County, AL",01019,16331.42


In [15]:
sunny_data = sunny.iloc[:3111].copy().drop('Notes', axis = 1)

In [16]:
sunny_data

,County,County Code,Avg Daily Sunlight (KJ/m²)
0,"Autauga County, AL",01001,17103.72
1,"Baldwin County, AL",01003,17660.59
2,"Barbour County, AL",01005,17315.13
3,"Bibb County, AL",01007,16904.87
4,"Blount County, AL",01009,16393.63
5,"Bullock County, AL",01011,17296.02
6,"Butler County, AL",01013,17323.69
7,"Calhoun County, AL",01015,16447.09
8,"Chambers County, AL",01017,16997.01
9,"Cherokee County, AL",01019,16331.42


In [17]:
sunny_data['state_id'] = sunny_data['County Code'].str.slice(stop=2)
sunny_data['county_id'] = sunny_data['County Code'].str.slice(start=2)

In [18]:
sunny_data

,County,County Code,Avg Daily Sunlight (KJ/m²),state_id,county_id
0,"Autauga County, AL",01001,17103.72,01,001
1,"Baldwin County, AL",01003,17660.59,01,003
2,"Barbour County, AL",01005,17315.13,01,005
3,"Bibb County, AL",01007,16904.87,01,007
4,"Blount County, AL",01009,16393.63,01,009
5,"Bullock County, AL",01011,17296.02,01,011
6,"Butler County, AL",01013,17323.69,01,013
7,"Calhoun County, AL",01015,16447.09,01,015
8,"Chambers County, AL",01017,16997.01,01,017
9,"Cherokee County, AL",01019,16331.42,01,019


In [19]:
# Create a map between state/county and avg daily sunlight
county2sunny = {(row['state_id'],row['county_id']):row['Avg Daily Sunlight (KJ/m²)']
                for i, row in sunny_data.iterrows()}

In [20]:
# We will need to scale the sunniness values to lie between 0 and 1
min_sun = min(county2sunny.values())
max_sun = max(county2sunny.values())
sun_range = max_sun - min_sun

In [21]:
from matplotlib.cm import coolwarm
from matplotlib.cm import Reds

def calculate_sunny(sun):
    # make sunniness a number between 0 and 1
    normalized_sun = (sun - min_sun) / sun_range
    
    # transform the sunniness to a matplotlib color
    mpl_color = Reds(normalized_sun)

    # transform from a matplotlib color to a valid CSS color
    gmaps_color = to_hex(mpl_color, keep_alpha=False)

    return gmaps_color

In [27]:
sun_colors = []

for feature in counties_geojson['features']:
    state = feature['properties']['STATE']
    county = feature['properties']['COUNTY']
    try:
        sun = county2sunny[(state,county)]
        color = calculate_sunny(sun)
    except KeyError:
        # no sun for that county: return default color
        color = (0, 0, 0, 0.3)
    sun_colors.append(color)

In [60]:
fig = gmaps.figure()
counties_layer = gmaps.geojson_layer(counties_geojson, fill_color=sun_colors, stroke_color=sun_colors,
                                     fill_opacity=0.8)
fig.add_layer(counties_layer)
install_locs = open_pv_w_loc2[['Lat','Lng']]
# hover_text = ["Size (kW): {}".format(row['size_kw']) for i, row in open_pv_w_loc2.iterrows()]
size = [int(x) for i, x in open_pv_w_loc2['size_normed'].iteritems()]
fig.add_layer(gmaps.symbol_layer(install_locs, fill_opacity=.5, stroke_opacity = .5,
                                 fill_color = 'blue',
                                 stroke_color = 'blue', scale = size))
fig

Figure(layout=FigureLayout(height='420px'))

In [61]:
open_pv_w_loc2.sort_values('size_kw', ascending=False).head()

,state,date_installed,incentive_prog_names,type,size_kw,appraised,zipcode,install_type,installer,cost_per_watt,...,State,State Code,Unk1,Unk2,Unk3,Unk4,Lat,Lng,Unk5,size_normed
54,TX,08/11/2016,NaN,uir,31191.00,NaN,75075,Residential,Axium Solar,NaN,...,Texas,TX,Collin,85.0,NaN,NaN,33.0250,-96.7397,4.0,11
113,TX,09/15/2017,NaN,uir,8791.35,NaN,75024,Commercial,Axium Solar,NaN,...,Texas,TX,Collin,85.0,NaN,NaN,33.0752,-96.7843,4.0,3
218,AL,01/04/2017,NaN,uir,3000.00,NaN,36912,Commercial,NaN,NaN,...,Alabama,AL,Choctaw,23.0,NaN,NaN,32.2177,-88.3234,4.0,1
839,IN,11/02/2018,NaN,uir,2721.00,NaN,47711,Commercial,"Morton Solar, LLC",1.3,...,Indiana,IN,Vanderburgh,163.0,NaN,NaN,38.0617,-87.5548,4.0,1
234,TX,01/06/2017,NaN,uir,1111.59,NaN,75211,Commercial,Axium Solar,NaN,...,Texas,TX,Dallas,113.0,NaN,NaN,32.7319,-96.9057,4.0,1


In [62]:
open_pv_w_loc2.loc[54].T

state                                           TX
date_installed                          08/11/2016
incentive_prog_names                           NaN
type                                           uir
size_kw                                      31191
appraised                                      NaN
zipcode                                      75075
install_type                           Residential
installer                              Axium Solar
cost_per_watt                                  NaN
cost                                           NaN
lbnl_tts_version_year                          NaN
lbnl_tts                                       NaN
city                                         Plano
utility_clean                                  NaN
tech_1                                         NaN
model1_clean                                   NaN
county                                         NaN
annual_PV_prod                                 NaN
annual_insolation              

In [28]:
open_pv_cluster = open_pv[open_pv['zipcode'].notnull()].copy()

In [29]:
open_pv_cluster2 = pd.merge(open_pv_cluster, zip_code_info, how = 'left', on = 'zipcode')

In [30]:
open_pv_cluster3 = open_pv_cluster2[open_pv_cluster2['Lat'].notnull() & open_pv_cluster2['Lng'].notnull()].copy()

In [31]:
open_pv_cluster4 = open_pv_cluster3[open_pv_cluster3['date_installed_dt'] > '1993'].copy()

In [32]:
# open_pv_cluster4 = open_pv_cluster3.copy()

In [33]:
open_pv_cluster4.shape

(1020545, 93)

In [35]:
from sklearn.cluster import MiniBatchKMeans
from datetime import datetime

print(datetime.now())

X = open_pv_cluster4[['Lat','Lng']]

kmeans = MiniBatchKMeans(n_clusters=1000, random_state=0, batch_size=100).fit(X)

print(datetime.now())

2019-04-05 16:31:26.400304


/anaconda3/lib/python3.7/site-packages/sklearn/cluster/k_means_.py:1559: RuntimeWarning: init_size=300 should be larger than k=1000. Setting it to 3*k
  init_size=init_size)
/anaconda3/lib/python3.7/site-packages/sklearn/cluster/k_means_.py:1559: RuntimeWarning: init_size=300 should be larger than k=1000. Setting it to 3*k
  init_size=init_size)
/anaconda3/lib/python3.7/site-packages/sklearn/cluster/k_means_.py:1559: RuntimeWarning: init_size=300 should be larger than k=1000. Setting it to 3*k
  init_size=init_size)


2019-04-05 16:31:34.826157


In [36]:
open_pv_cluster4['new_loc_ind'] = kmeans.predict(X)

In [37]:
new_locs = pd.DataFrame(kmeans.cluster_centers_, columns = ['new_lat','new_lng'])

In [38]:
open_pv_cluster5 = pd.merge(open_pv_cluster4, new_locs, how = 'left', left_on = 'new_loc_ind', right_index = True)

In [39]:
open_pv_cluster5.head()

,state,date_installed,incentive_prog_names,type,size_kw,appraised,zipcode,install_type,installer,cost_per_watt,...,Unk1,Unk2,Unk3,Unk4,Lat,Lng,Unk5,new_loc_ind,new_lat,new_lng
0,TX,12/12/2013,Austin Energy,uir,10.560,NaN,78702,residential,NaN,2.897777,...,Travis,453.0,NaN,NaN,30.2638,-97.7166,4.0,543,30.28405,-97.6632
1,TX,07/11/2014,Austin Energy,uir,12.720,NaN,78702,residential,NaN,3.144654,...,Travis,453.0,NaN,NaN,30.2638,-97.7166,4.0,543,30.28405,-97.6632
2,TX,01/17/2014,Austin Energy,uir,5.400,NaN,78702,residential,NaN,4.197330,...,Travis,453.0,NaN,NaN,30.2638,-97.7166,4.0,543,30.28405,-97.6632
3,IN,10/06/2017,NaN,uir,6.785,NaN,47403,Residential,Whole Sun Designs,2.590000,...,Monroe,105.0,NaN,NaN,39.1263,-86.5769,4.0,108,38.17060,-87.0070
4,IN,10/06/2017,NaN,uir,4.720,NaN,47408,Residential,Whole Sun Designs,2.570000,...,Monroe,105.0,NaN,NaN,39.2303,-86.4692,4.0,108,38.17060,-87.0070


In [40]:
open_pv_cluster6 = open_pv_cluster5.groupby(['new_loc_ind','new_lat','new_lng'])['size_kw'].sum().reset_index()

In [41]:
open_pv_cluster6.shape

(977, 4)

In [42]:
min_size = open_pv_cluster6['size_kw'].min()
max_size = open_pv_cluster6['size_kw'].max()
size_range = max_size - min_size
open_pv_cluster6['size_normed'] = (10 * (open_pv_cluster6['size_kw'] - min_size)/size_range + 1).astype(int)

In [43]:
fig = gmaps.figure()
counties_layer = gmaps.geojson_layer(counties_geojson, fill_color=sun_colors, stroke_color=sun_colors,
                                     fill_opacity=0.8)
fig.add_layer(counties_layer)
install_locs = open_pv_cluster6[['new_lat','new_lng']]
# hover_text = ["Size (kW): {}".format(row['size_kw']) for i, row in open_pv_w_loc2.iterrows()]
size = [int(x) for i, x in open_pv_cluster6['size_normed'].iteritems()]
fig.add_layer(gmaps.symbol_layer(install_locs, fill_opacity=.5, stroke_opacity = .5,
                                 fill_color = 'blue',
                                 stroke_color = 'blue', scale = size))
fig

Figure(layout=FigureLayout(height='420px'))